데이터 다운받고 zip파일 추출하기, callback, **Image Generator** 사용

In [14]:
import tensorflow as tf
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
DESIRED_ACC = 0.999

### Load data

In [3]:
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" -O "/tmp/happy-or-sad.zip"

--2021-01-23 18:13:05--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 34.64.4.80, 34.64.4.16, 2404:f340:10:1802::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|34.64.4.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  7.88MB/s    in 0.3s    

2021-01-23 18:13:06 (7.88 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [4]:
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

### Callback

In [5]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>DESIRED_ACC):
            print("\n\nEnough accuracy so cancel training")
            self.model.stop_training=True

In [6]:
callbacks = myCallback()

### 데이터 보기

In [8]:
os.listdir('/tmp/h-or-s')

['sad', 'happy']

In [9]:
os.listdir('/tmp/h-or-s/happy')

['happy2-15.png',
 'happy2-09.png',
 'happy1-14.png',
 'happy2-08.png',
 'happy2-10.png',
 'happy1-04.png',
 'happy1-00.png',
 'happy1-03.png',
 'happy1-08.png',
 'happy1-15.png',
 'happy1-09.png',
 'happy2-06.png',
 'happy1-13.png',
 'happy2-17.png',
 'happy2-01.png',
 'happy2-02.png',
 'happy1-18.png',
 'happy1-19.png',
 'happy1-05.png',
 'happy2-18.png',
 'happy2-07.png',
 'happy2-12.png',
 'happy2-13.png',
 'happy2-16.png',
 'happy2-03.png',
 'happy1-07.png',
 'happy1-10.png',
 'happy1-02.png',
 'happy1-01.png',
 'happy1-16.png',
 'happy2-11.png',
 'happy1-17.png',
 'happy1-12.png',
 'happy2-04.png',
 'happy2-05.png',
 'happy2-14.png',
 'happy2-00.png',
 'happy1-06.png',
 'happy2-19.png',
 'happy1-11.png']

In [17]:
train_dir = os.path.join('/tmp/h-or-s/')

In [10]:
# 데이터 개수 추출
train_name_happy = os.listdir('/tmp/h-or-s/happy')
train_name_sad = os.listdir('/tmp/h-or-s/sad')
print(len(train_name_happy), len(train_name_sad))

40 40


In [12]:
# 데이터 사이즈 추출
get_name = train_name_happy[0]
get_path = os.path.join('/tmp/h-or-s/happy', get_name)
get_path

'/tmp/h-or-s/happy/happy2-15.png'

In [13]:
from PIL import Image
get_img = Image.open(get_path)
get_img.size

(150, 150)

참고로 굳이 이미지 데이터의 사이즈를 추출할 필요는 없다. `ImageDataGenerator`를 쓸 것이기 때문에 `target_size`로 이미지 사이즈를 지정할 것이기 때문.

### Preprocessing : ImageDataGenerator
`from tensorflow.keras.preprocessing.image import ImageDataGenerator`

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size=8,
    class_mode = 'binary'
)

Found 80 images belonging to 2 classes.


### Modeling

In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
# model fitting
history = model.fit(
    train_generator,
    steps_per_epoch = 10, #데이터 개수 = batch_size * steps_per_epoch
    epochs=20,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/20
 9/10 [==========================>...] - ETA: 0s - loss: 0.0501 - accuracy: 1.0000

Enough accuracy so cancel training
10/10 [==============================] - 0s 46ms/step - loss: 0.0451 - accuracy: 1.0000


### Evaluate